# Kundt’s Tube Speed of Sound Notebook
In this notebook, we estimate the speed of sound in air using resonance data from a Kundt’s tube.

## How to Use
1. **Edit** the "User Inputs" in the second cell:
   - Meter-stick readings (in cm)
   - Observed resonance frequencies (list of floats)
   - Plot title
2. **Run** that cell.
3. **Run** the final cell to perform the analysis.

You will get:
- A table listing the assigned harmonic numbers, observed frequencies, and calculated frequencies.
- A printed speed of sound.
- A plot displaying both observed data and the linear regression, saved as a PNG file.
- The regression line equation (forced intercept = 0) and the computed R² on the plot.

In [ ]:
# ---------------------------------------------------------
# User Inputs
# ---------------------------------------------------------

# 1) Meter-stick readings (in centimeters)
#    Example: Tube from 12.3 cm to 46.3 cm -> 34 cm length
length_start_cm = 12.3
length_end_cm   = 46.3

# 2) Observed resonance frequencies (in any order). Possibly missing some modes.
observed_freqs = [5050, 723, 1449, 3608, 2890, 4403, 2165]

# 3) Plot title
plot_title = "Kundt’s Tube Resonances: Observed vs. Regression"

# (Run this cell after making changes to these values.)
print("User inputs have been set. Proceed to run the next cell.")

In [ ]:
# ---------------------------------------------------------
# Main Analysis and Plotting
# ---------------------------------------------------------

import numpy as np
import matplotlib.pyplot as plt

def forced_zero_intercept_linear_regression(x_vals, y_vals):
    """
    Returns slope 'm' from a forced-zero-intercept fit y = m*x.
    Formula: m = sum(x*y) / sum(x^2).
    """
    x_arr = np.array(x_vals, dtype=float)
    y_arr = np.array(y_vals, dtype=float)
    m = np.sum(x_arr * y_arr) / np.sum(x_arr * x_arr)
    return m

def compute_r_squared_forced_intercept(x_vals, y_vals, slope):
    """
    Computes R^2 for a forced-zero intercept regression.
    We'll still use the standard R^2 definition:
        R^2 = 1 - SSE / SST,
    where SSE = sum( (y_i - (m*x_i))^2 ),
          SST = sum( (y_i - mean(y))^2 ).
    """
    x_arr = np.array(x_vals, dtype=float)
    y_arr = np.array(y_vals, dtype=float)
    # Predicted y from forced intercept
    y_pred = slope * x_arr

    sse = np.sum((y_arr - y_pred)**2)
    sst = np.sum((y_arr - np.mean(y_arr))**2)

    if sst == 0:
        return 1.0 if sse == 0 else 0.0
    return 1.0 - (sse / sst)

# ---------------------------------------------------------
# 1) Compute tube length in meters.
# ---------------------------------------------------------
L_cm = length_end_cm - length_start_cm
L_m  = L_cm / 100.0  # convert cm -> m

# ---------------------------------------------------------
# 2) Sort the observed frequencies.
# ---------------------------------------------------------
observed_freqs.sort()

# ---------------------------------------------------------
# 3) First pass: Assume these are n=1,2,3,... in ascending order.
#    Perform forced-zero intercept regression to get initial slope.
# ---------------------------------------------------------
x_pass1 = np.arange(1, len(observed_freqs) + 1, 1, dtype=float)
y_pass1 = np.array(observed_freqs, dtype=float)
slope_pass1 = forced_zero_intercept_linear_regression(x_pass1, y_pass1)

# ---------------------------------------------------------
# 4) Second pass: Assign n_i = round(f_i / slope_pass1), then re-fit.
# ---------------------------------------------------------
n_guess = []
for f in observed_freqs:
    n_i = int(round(f / slope_pass1))
    if n_i < 1:
        n_i = 1  # avoid non-physical n=0
    n_guess.append(n_i)

n_guess = np.array(n_guess, dtype=float)
f_data  = np.array(observed_freqs, dtype=float)

slope_final = forced_zero_intercept_linear_regression(n_guess, f_data)

# ---------------------------------------------------------
# 5) Speed of sound and fundamental frequency.
#    slope_final = v/(2L) => v = slope_final * 2L
# ---------------------------------------------------------
speed_of_sound = slope_final * 2.0 * L_m
fundamental_freq_est = slope_final  # slope = f1, for n=1

# ---------------------------------------------------------
# 6) Prepare table of (n, f_obs, f_calc) and note missing n.
# ---------------------------------------------------------
sort_idx = np.argsort(n_guess)
n_int = n_guess[sort_idx].astype(int)
f_obs_sorted  = f_data[sort_idx]
f_calc_sorted = slope_final * n_int

full_n_range = np.arange(n_int.min(), n_int.max() + 1)
missing_n = [ni for ni in full_n_range if ni not in n_int]

print("Assigned Harmonic Numbers (after second pass):")
print("  n   f_obs [Hz]   f_calc [Hz]")
for i in range(len(n_int)):
    print(f"{n_int[i]:3d}   {f_obs_sorted[i]:9.2f}   {f_calc_sorted[i]:10.2f}")
if missing_n:
    print("\nMissing modes (no measured freq):", missing_n)
else:
    print("\nNo missing integer modes in this sequence.")

# ---------------------------------------------------------
# 7) Compute R^2 for the final regression.
# ---------------------------------------------------------
r2_final = compute_r_squared_forced_intercept(n_int, f_obs_sorted, slope_final)

print(f"\nExtracted slope (final regression) = {slope_final:.3f} Hz/mode")
print(f"Tube length = {L_cm:.2f} cm => {L_m:.3f} m")
print(f"Estimated fundamental frequency (n=1) = {fundamental_freq_est:.2f} Hz")
print(f"Speed of sound = {speed_of_sound:.2f} m/s")
print(f"R^2 = {r2_final:.4f}")

# ---------------------------------------------------------
# 8) Plot:
#    - Observed freq vs. assigned n
#    - Best-fit line
#    - Only plot "calculated from final slope" for missing n
#    - Annotate equation f(n) = slope * n, R^2, L, f1, v
#    - Save figure
# ---------------------------------------------------------

plt.figure(figsize=(7,5))

# Observed data (blue dots)
plt.scatter(n_int, f_obs_sorted, color='blue', label='Observed frequencies')

# Best-fit line
n_plot = np.linspace(n_int.min(), n_int.max(), 200)
f_plot = slope_final * n_plot
plt.plot(n_plot, f_plot, color='red', label='Regression line')

# Plot only missing n in orange
if missing_n:
    missing_n_arr = np.array(missing_n, dtype=float)
    f_missing_calc = slope_final * missing_n_arr
    plt.scatter(missing_n_arr, f_missing_calc, color='orange', marker='x',
                label='Calculated for missing modes')

plt.xlabel('Harmonic number n')
plt.ylabel('Frequency [Hz]')
plt.title(plot_title)
plt.grid(True)
plt.legend(loc='upper right')

# Annotate text
equation_str = f"f(n) = {slope_final:.2f}·n"
text_str = (f"{equation_str}\n"
            f"R² = {r2_final:.3f}\n"
            f"Tube length L = {L_m:.3f} m\n"
            f"Fundamental f₁ ≈ {fundamental_freq_est:.1f} Hz\n"
            f"Speed of sound v ≈ {speed_of_sound:.1f} m/s")

plt.text(0.05, 0.95, text_str, transform=plt.gca().transAxes,
         fontsize=10, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

plt.tight_layout()

png_filename = "kundts_tube_resonances.png"
plt.savefig(png_filename, dpi=300)
print(f"\nFigure saved as {png_filename}")

plt.show()
